In [1]:

import os
from datetime import datetime

# Project paths (mounted volumes are recommended in Docker)
PIPELINE_NAME = "poc_ch3_5"
BASE_DIR = os.path.abspath("../../")  # change to a mounted volume e.g., /workspace
DATA_DIR = os.path.join(BASE_DIR, "data")
PIPELINE_OUTPUT_BASE = os.path.join(BASE_DIR, "artifacts")
PIPELINE_ROOT = os.path.join(PIPELINE_OUTPUT_BASE, PIPELINE_NAME)
METADATA_PATH = os.path.join(PIPELINE_ROOT, "metadata", "metadata.db")
SERVING_MODEL_DIR = os.path.join(PIPELINE_ROOT, "serving_model")

for p in [DATA_DIR, PIPELINE_ROOT, SERVING_MODEL_DIR]:
    os.makedirs(p, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("DATA_DIR:", DATA_DIR)
print("PIPELINE_ROOT:", PIPELINE_ROOT)
print("METADATA_PATH:", METADATA_PATH)


BASE_DIR: /
DATA_DIR: /data
PIPELINE_ROOT: /artifacts/poc_ch3_5
METADATA_PATH: /artifacts/poc_ch3_5/metadata/metadata.db


In [2]:
from tfx import v1 as tfx
from tfx.components import (
    ExampleValidator,
    SchemaGen,
    StatisticsGen,
    Transform,
)
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.example_gen.custom_executors import parquet_executor
from tfx.components.base import executor_spec
from tfx.proto import example_gen_pb2
from tfx.proto import transform_pb2
from tfx.dsl.components.base import executor_spec
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import FileBasedExampleGen
from tfx.components.example_gen.custom_executors import parquet_executor
# from tfx.utils.dsl_utils import external_input

import tensorflow as tf
import tensorflow_transform as tft

import tensorflow_data_validation as tfdv
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

import pprint, pathlib, glob, json
print("TFX version:", tfx.__version__)
print("TF version:", tf.__version__)

2025-09-20 16:22:49.916328: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-20 16:22:49.916786: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-20 16:22:49.921439: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-20 16:22:49.960127: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-20 16:22:50.847274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

TFX version: 1.16.0
TF version: 2.16.1


### Interactive Context

In [3]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)
print("InteractiveContext ready.")

InteractiveContext ready.


### 1- Exmple Gen (Parquet) __ Ingestion& Split

In [4]:
output_config  = example_gen_pb2.Output(
     split_config=example_gen_pb2.SplitConfig(splits=[
     example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
     example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)]))

example_gen = FileBasedExampleGen(
    input_base="../../data/",
    input_config=example_gen_pb2.Input(splits=[
        example_gen_pb2.Input.Split(name='train', pattern='*.parquet'),
    ]),
    output_config=output_config,
    custom_executor_spec=executor_spec.ExecutorClassSpec(parquet_executor.Executor),
)

context.run(example_gen)

ExecutionResult(
    component_id: FileBasedExampleGen
    execution_id: 31
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=FileBasedExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### visualize the examples 

In [5]:
import os, glob, tensorflow as tf
import pandas as pd
from tfx.types import artifact_utils

def preview_examples(example_channel, split="train", n=5):
    # 1) Get the list of materialized Example artifacts
    artifacts = example_channel.get()  # <-- important: materialize channel -> list[Artifact]

    # 2) Resolve the URI for the requested split
    split_uri = artifact_utils.get_split_uri(artifacts, split)
    print("Split URI:", split_uri)

    # 3) Grab a TFRecord file (they’re usually GZIP’d)
    candidates = sorted(glob.glob(os.path.join(split_uri, "*.gz")))
    if not candidates:
        # some versions write into a nested dir like 'train/*'
        candidates = sorted(glob.glob(os.path.join(split_uri, "**", "*.gz"), recursive=True))
    if not candidates:
        raise FileNotFoundError(f"No TFRecord .gz files found under {split_uri}")
    tfrecord_path = candidates[0]
    print("Reading:", tfrecord_path)

    # 4) Read a few examples
    ds = tf.data.TFRecordDataset(tfrecord_path, compression_type="GZIP")
    rows = []
    for raw in ds.take(n):
        ex = tf.train.Example()
        ex.ParseFromString(raw.numpy())
        # Convert to a flat dict (bytes/float/int lists)
        as_dict = {
            k: (list(f.bytes_list.value) or
                list(f.float_list.value) or
                list(f.int64_list.value))
            for k, f in ex.features.feature.items()
        }
        rows.append(as_dict)

    df = pd.DataFrame(rows)
    return df

# Use it:
df_head = preview_examples(example_gen.outputs["examples"], split="train", n=5)
df_head


Split URI: /artifacts/poc_ch3_5/FileBasedExampleGen/examples/31/Split-train
Reading: /artifacts/poc_ch3_5/FileBasedExampleGen/examples/31/Split-train/data_tfrecord-00000-of-00001.gz


2025-09-20 16:25:17.307907: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,isFraud,nameDest,step,nameOrig,type,isFlaggedFraud,amount,oldbalanceOrg,newbalanceDest,newbalanceOrig,oldbalanceDest
0,[],[b'M1979787155'],[1],[b'C1231006815'],[b'PAYMENT'],[],[9839.6396484375],[170136.0],[],[160296.359375],[]
1,[],[b'M2044282225'],[1],[b'C1666544295'],[b'PAYMENT'],[],[1864.280029296875],[21249.0],[],[19384.720703125],[]
2,[1],[b'C38997010'],[1],[b'C840083671'],[b'CASH_OUT'],[],[181.0],[181.0],[],[],[21182.0]
3,[],[b'M1230701703'],[1],[b'C2048537720'],[b'PAYMENT'],[],[11668.1396484375],[41554.0],[],[29885.859375],[]
4,[],[b'M573487274'],[1],[b'C90045638'],[b'PAYMENT'],[],[7817.7099609375],[53860.0],[],[46042.2890625],[]


#### Spliting the data alternatives

In [ ]:


output = example_gen_pb2.Output(
     split_config=example_gen_pb2.SplitConfig(splits=[
     example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
     example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
     example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)]))

### 2- StatisticsGen __ Data Profiling

In [7]:
def generate_stats_from_examples(example_channel, split="train"):
    split_uri = artifact_utils.get_split_uri(example_channel.get(), split)
    filepattern = os.path.join(split_uri, "data_tfrecord-*")  # covers sharded outputs
    print("Using:", filepattern)
    return tfdv.generate_statistics_from_tfrecord(
        data_location=filepattern  # avoid tf_record_iterator warning
    )

# Example: generate stats from train split
train_stats = generate_stats_from_examples(example_gen.outputs["examples"], "train")
tfdv.visualize_statistics(train_stats)

Using: /artifacts/poc_ch3_5/FileBasedExampleGen/examples/31/Split-train/data_tfrecord-*
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [8]:
from tfx.components import StatisticsGen
statistics_gen = StatisticsGen(
 examples=example_gen.outputs['examples'])
context.run(statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 32
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [9]:
context.show(statistics_gen.outputs['statistics'])


## SchemaGen

In [10]:
schema_gen = SchemaGen(
 statistics=statistics_gen.outputs['statistics'],
 infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 33
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [12]:
context.show(schema_gen)

SchemaGen(spec: <tfx.types.standard_component_specs.SchemaGenSpec object at 0x76dc26feebc0>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x76dc26fef610>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: SchemaGen, inputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}, outputs: {'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)})

In [13]:
example_validator = ExampleValidator(
 statistics=statistics_gen.outputs['statistics'],
 schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 34
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [15]:
# import tensorflow_data_validation as tfdv

# tfdv.display_schema(schema)

In [16]:
from tfx.types import artifact_utils
from tfx.utils import io_utils
from tensorflow_metadata.proto.v0 import schema_pb2
import os

# 1) Resolve the artifacts from the OutputChannel
schema_artifacts = schema_gen.outputs['schema'].get()
assert len(schema_artifacts) == 1, f"Expected 1 schema artifact, got {len(schema_artifacts)}"
schema_uri = schema_artifacts[0].uri
schema_path = os.path.join(schema_uri, 'schema.pbtxt')

# 2) Load the schema proto
schema = schema_pb2.Schema()
io_utils.parse_pbtxt_file(schema_path, schema)

# 3) Inspect
print("Features:", [f.name for f in schema.feature])
for f in schema.feature:
    print(f.name, schema_pb2.FeatureType.Name(f.type))

Features: ['amount', 'isFlaggedFraud', 'isFraud', 'nameDest', 'nameOrig', 'newbalanceDest', 'newbalanceOrig', 'oldbalanceDest', 'oldbalanceOrg', 'step', 'type']
amount FLOAT
isFlaggedFraud BYTES
isFraud INT
nameDest BYTES
nameOrig BYTES
newbalanceDest FLOAT
newbalanceOrig FLOAT
oldbalanceDest FLOAT
oldbalanceOrg FLOAT
step INT
type BYTES


In [17]:
import tensorflow_data_validation as tfdv
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'amount',FLOAT,required,,-
'isFlaggedFraud',BYTES,optional,,-
'isFraud',INT,optional,single,-
'nameDest',STRING,required,,'nameDest'
'nameOrig',STRING,required,,'nameOrig'
'newbalanceDest',FLOAT,optional,single,-
'newbalanceOrig',FLOAT,optional,single,-
'oldbalanceDest',FLOAT,optional,single,-
'oldbalanceOrg',FLOAT,required,,-


,Values
Domain,
'nameDest',"'C1100439041', 'C195600860', 'C38997010', 'C476402209', 'M1230701703', 'M1635378213', 'M1731217984', 'M1877062907', 'M1979787155', 'M2044282225', 'M2096539129', 'M408069119', 'M473053293', 'M573487274', 'M972865270', 'C553264065', 'C997608398', 'M1176932104', 'M633326333', 'M801569151'"
'nameOrig',"'C1026483832', 'C1231006815', 'C1237762639', 'C154988899', 'C1648232591', 'C1666544295', 'C1670993182', 'C2033524545', 'C2048537720', 'C249177573', 'C712410124', 'C761750706', 'C840083671', 'C90045638', 'C905080434', 'C1265012928', 'C1305486145', 'C1716932897', 'C1900366749', 'C1912850431'"
'type',"'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER'"


In [ ]:
tfdv.visualize_statistics(

### Data Transformation

In [18]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.join("./", "preprocessing.py")  # we will write this module next
)

#### processing.py to_change later 

In [27]:
preprocessing_module_code = r'''
# preprocessing.py (fixed)
# preprocessing.py (cast integer outputs to int64)
# preprocessing.py (robust scalarization for varlen features)
# preprocessing.py (final, reduced + robust)
import tensorflow as tf
import tensorflow_transform as tft

# ======================
# Columns to keep
# ======================
LABEL_KEY = 'isFraud'
NUMERIC_FLOAT_KEYS = [
    'amount',
    'oldbalanceOrg',
    'newbalanceOrig',
    'oldbalanceDest',
    'newbalanceDest',
]
_XF = '_xf'
ZSCORE_CLIP = 5.0


# ======================
# Robust scalarization
# ======================
def _first_with_default_ragged(rt: tf.RaggedTensor, default_value, out_dtype):
    """First element per row; empty-row -> default; handles global-empty safely."""
    batch_size = tf.shape(rt.row_splits)[0] - 1  # N rows
    total_vals = tf.shape(rt.flat_values)[0]

    def all_empty_case():
        return tf.fill([batch_size], tf.cast(default_value, out_dtype))

    def normal_case():
        splits = rt.row_splits  # [N+1]
        starts = splits[:-1]
        lens = splits[1:] - splits[:-1]
        has = tf.not_equal(lens, 0)
        safe_idx = tf.where(has, starts, tf.zeros_like(starts))
        gathered = tf.gather(rt.flat_values, safe_idx)  # safe (flat_values > 0)
        gathered = tf.cast(gathered, out_dtype)
        default_tensor = tf.cast(default_value, out_dtype)
        return tf.where(has, gathered, tf.fill([batch_size], default_tensor))

    return tf.cond(total_vals > 0, normal_case, all_empty_case)


def _scalar_from_any(x, default_value, out_dtype):
    """Convert Sparse/Ragged/Dense (possibly varlen) to shape [batch] scalar tensor."""
    # Sparse -> Ragged
    if isinstance(x, tf.SparseTensor):
        return _first_with_default_ragged(tf.RaggedTensor.from_sparse(x), default_value, out_dtype)

    # Ragged
    if isinstance(x, tf.RaggedTensor):
        return _first_with_default_ragged(x, default_value, out_dtype)

    # Dense
    x = tf.convert_to_tensor(x)
    # If rank==1, already [batch]
    if x.shape.rank == 1:
        return tf.cast(x, out_dtype)

    # General dense fallback: choose first column if exists, otherwise defaults
    batch = tf.shape(x)[0]
    width = tf.shape(x)[-1]

    def empty_case():
        return tf.fill([batch], tf.cast(default_value, out_dtype))

    def nonempty_case():
        # If it's [batch, 1] this also works
        return tf.cast(x[:, 0], out_dtype)

    return tf.cond(width > 0, nonempty_case, empty_case)


def _to_float_scalar(x, default=0.0):
    t = _scalar_from_any(x, default_value=default, out_dtype=tf.float32)
    # If string, coerce to number
    if t.dtype == tf.string:
        s = tf.where(tf.equal(t, b''), tf.constant(str(default).encode()), t)
        t = tf.strings.to_number(s, out_type=tf.float32)
    else:
        t = tf.cast(t, tf.float32)
    return t


def _to_int64_scalar(x, default=0):
    t = _scalar_from_any(x, default_value=default, out_dtype=tf.int64)
    if t.dtype == tf.string:
        s = tf.where(tf.equal(t, b''), tf.constant(str(default).encode()), t)
        f = tf.strings.to_number(s, out_type=tf.float32)
        t = tf.cast(tf.round(f), tf.int64)
    elif t.dtype.is_floating:
        t = tf.cast(tf.round(t), tf.int64)
    else:
        t = tf.cast(t, tf.int64)
    return t


def _clip_z(x, clip=ZSCORE_CLIP):
    return tf.clip_by_value(x, -clip, clip)


# ======================
# Main preprocessing
# ======================
def preprocessing_fn(inputs):
    outputs = {}

    # Label -> int64 {0,1}
    if LABEL_KEY in inputs:
        label = _to_int64_scalar(inputs[LABEL_KEY], default=0)
        label = tf.clip_by_value(label, 0, 1)
        outputs[LABEL_KEY + _XF] = tf.cast(label, tf.int64)

    # Selected numeric features -> float32 z-score (clipped)
    for k in NUMERIC_FLOAT_KEYS:
        if k in inputs:
            v = _to_float_scalar(inputs[k], default=0.0)
            v = tft.scale_to_z_score(v)
            v = _clip_z(v)
            outputs[k + _XF] = v

    return outputs

'''
with open(os.path.join("./", "preprocessing.py"), "w", encoding="utf-8") as f:
    f.write(preprocessing_module_code)

print("Wrote preprocessing module:", os.path.join(BASE_DIR, "preprocessing.py"))

Wrote preprocessing module: /preprocessing.py


In [29]:
context.run(transform)
print("Transform outputs:")
# print("   Transform graph:", tfx.types.artifact_utils.get_single_uri(transform.outputs['transform_graph']))
# print("   Transformed examples:", tfx.types.artifact_utils.get_single_uri(transform.outputs['transformed_examples']))

Transform outputs:


In [31]:
print(transform.outputs['transform_graph'].get()[0].uri)
print(transform.outputs['transformed_examples'].get()[0].uri)

/artifacts/poc_ch3_5/Transform/transform_graph/39
/artifacts/poc_ch3_5/Transform/transformed_examples/39


In [33]:
import tensorflow as tf
import glob, os
import tensorflow_transform as tft

te_uri = "/artifacts/poc_ch3_5/Transform/transformed_examples/39"
tg_uri = "/artifacts/poc_ch3_5/Transform/transform_graph/39"
tfmd = tft.TFTransformOutput(tg_uri)
t_spec = tfmd.transformed_feature_spec()

LABEL = "isFraud_xf"

def _parse_fn(serialized):
    example = tf.io.parse_single_example(serialized, t_spec)
    y = tf.cast(example.pop(LABEL), tf.float32)  # for binary Keras
    return example, y

def make_ds(split_name="train", batch=1024, shuffle=True):
    pattern = os.path.join(te_uri, f"Split-{split_name}", "*.gz")
    files = tf.io.gfile.glob(pattern)
    ds = tf.data.TFRecordDataset(files, compression_type="GZIP")
    if shuffle:
        ds = ds.shuffle(10000, reshuffle_each_iteration=True)
    ds = ds.map(_parse_fn, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_ds("train")
eval_ds  = make_ds("eval", shuffle=False)

# peek
for (features, labels) in train_ds.take(1):
    print({k: v.shape for k,v in features.items()}, labels.shape)

{'amount_xf': TensorShape([15]), 'newbalanceDest_xf': TensorShape([15]), 'newbalanceOrig_xf': TensorShape([15]), 'oldbalanceDest_xf': TensorShape([15]), 'oldbalanceOrg_xf': TensorShape([15])} (15,)


2025-09-20 16:49:01.313246: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### visulaize

In [34]:
import pandas as pd

def ds_to_df(ds, n_batches=1):
    rows = []
    for features, labels in ds.take(n_batches):
        # Convert each batch element into a row
        f_np = {k: v.numpy() for k, v in features.items()}
        y_np = labels.numpy()
        for i in range(len(y_np)):
            row = {k: f_np[k][i] for k in f_np}
            row[LABEL] = y_np[i]
            rows.append(row)
    return pd.DataFrame(rows)

# Example: take first batch of train split
df_sample = ds_to_df(train_ds, n_batches=1)
print(df_sample.head())

   amount_xf  newbalanceDest_xf  newbalanceOrig_xf  oldbalanceDest_xf  \
0  -0.408903          -0.388932          -0.397310          -0.500293   
1  -0.293380          -0.388932          -0.301122          -0.500293   
2  -0.435088          -0.388932          -0.378984          -0.500293   
3  -0.318033          -0.388932           0.725823          -0.500293   
4  -0.354867          -0.388932           0.850172          -0.500293   

   oldbalanceOrg_xf  isFraud_xf  
0         -0.408678         0.0  
1         -0.245799         0.0  
2         -0.405661         0.0  
3          0.761918         0.0  
4          0.864264         0.0  


2025-09-20 16:49:39.929352: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Train

In [35]:
import tensorflow as tf

LABEL = "isFraud_xf"

def to_tfdf_xy(features, label):
    # TF-DF classification prefers int labels
    return features, tf.cast(label, tf.int32)

train_ds_rf = train_ds.map(to_tfdf_xy, num_parallel_calls=tf.data.AUTOTUNE)
eval_ds_rf  = eval_ds.map(to_tfdf_xy,  num_parallel_calls=tf.data.AUTOTUNE)

In [36]:
import tensorflow_decision_forests as tfdf
import tf_keras as keras  # <- important on TF 2.16+

rf = tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.CLASSIFICATION,
    num_trees=300,
    max_depth=16,
    random_seed=123,
    verbose=1,
)

history = rf.fit(
    train_ds_rf,
    validation_data=eval_ds_rf,   # optional; shows eval metrics in Keras history
)

ModuleNotFoundError: No module named 'tensorflow_decision_forests'